In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal, List
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

# -----------------------------
# 1. Define the shared state
# -----------------------------
class AssistantState(TypedDict):
    user_input: str
    intent: Literal["generate", "explain", "unknown"]
    relevant_examples: List[str]
    llm_response: str
    final_output: str

In [ ]:

# -----------------------------
# 2. Load model
# -----------------------------
model_id = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# -----------------------------
# 3. Node: Intent Classifier
# -----------------------------
def classify_intent(state: AssistantState) -> AssistantState:
    user_input = state["user_input"].lower()
    generate_keywords = ["create", "write", "generate", "build", "make", "code", "function"]
    explain_keywords = ["explain", "what does", "how does", "understand", "clarify"]

    if any(k in user_input for k in generate_keywords):
        state["intent"] = "generate"
    elif any(k in user_input for k in explain_keywords):
        state["intent"] = "explain"
    else:
        state["intent"] = "unknown"
    return state

In [ ]:
# -----------------------------
# 4. Node: Retrieve Examples
# -----------------------------
example_bank = {
    "generate": [
        "def fibonacci(n):\n    if n <= 1:\n        return n\n    return fibonacci(n-1) + fibonacci(n-2)"
    ],
    "explain": [
        "This function returns the Fibonacci number at index n using recursion."
    ]
}
def retrieve_examples(state: AssistantState) -> AssistantState:
    state["relevant_examples"] = example_bank.get(state["intent"], [])
    return state

In [ ]:
# -----------------------------
# 5. Node: LLM Call (Updated)
# -----------------------------
def call_llm(state: AssistantState) -> AssistantState:
    user_input = state["user_input"].strip()
    prompt = f"[INST] {user_input} [/INST]"
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    try:
        output_ids = model.generate(
            input_ids,
            max_new_tokens=256,
            do_sample=False,  # Deterministic output
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
        output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        response = output_text.split("[/INST]")[-1].strip()

        # 🧠 Trim to first meaningful block
        if state["intent"] == "generate":
            match = re.search(r"(def .*?:\n(?:\s+.+\n*)+)", response)
            if match:
                response = match.group(1).strip()
        elif state["intent"] == "explain":
            response = response.split("\n\n")[0].strip()

        if not response:
            response = "⚠️ The model did not return a valid response. Try rephrasing."

        state["llm_response"] = response

    except Exception as e:
        state["llm_response"] = f"❌ Error: {str(e)}"

    return state

In [ ]:
# -----------------------------
# 6. Node: Format Output
# -----------------------------
def format_output(state: AssistantState) -> AssistantState:
    if state["intent"] == "generate":
        state["final_output"] = f"🛠️ Generated Python Code:\n\n{state['llm_response']}"
    elif state["intent"] == "explain":
        state["final_output"] = f"📘 Explanation:\n\n{state['llm_response']}"
    else:
        state["final_output"] = f"💬 Response:\n\n{state['llm_response'] or 'Sorry, I couldn’t understand your request. Try rephrasing it.'}"
    return state

In [ ]:
# -----------------------------
# 7. Build LangGraph Workflow
# -----------------------------
builder = StateGraph(AssistantState)

builder.add_node("classify_intent", classify_intent)
builder.add_node("retrieve_examples", retrieve_examples)
builder.add_node("call_llm", call_llm)
builder.add_node("format_output", format_output)

builder.set_entry_point("classify_intent")
builder.add_edge("classify_intent", "retrieve_examples")
builder.add_edge("retrieve_examples", "call_llm")
builder.add_edge("call_llm", "format_output")
builder.add_edge("format_output", END)

graph = builder.compile()

In [10]:


# -----------------------------
# 8. CLI Interaction
# -----------------------------
def run_assistant():
    print("🚀 LangGraph Python Assistant is ready!")
    while True:
        user_input = input("\n🧑‍💻 Ask your Python question (or type 'exit' to quit):\n> ").strip()
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("\n👋 Goodbye! Have a great day.")
            break

        initial_state: AssistantState = {
            "user_input": user_input,
            "intent": "unknown",
            "relevant_examples": [],
            "llm_response": "",
            "final_output": ""
        }

        final_state = graph.invoke(initial_state)
        print(f"\n{final_state['final_output']}")

# -----------------------------
# 9. Entry Point
# -----------------------------
if __name__ == "__main__":
    run_assistant()


🚀 LangGraph Python Assistant is ready!

🧑‍💻 Ask your Python question (or type 'exit' to quit):
> Write Python code to build list func


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🛠️ Generated Python Code:

def build_list(n, m):
    list_func = []
    for i in range(n, m+1):
        list_func.append(i)
    return list_func

🧑‍💻 Ask your Python question (or type 'exit' to quit):
> explain list func


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



📘 Explanation:

[INST] explain list

🧑‍💻 Ask your Python question (or type 'exit' to quit):
> explain list function


...


💬 Response:

List comprehension is a compact way of creating a new list by performing some operation on each item in an existing list. It is a more Pythonic way of writing list comprehensions.

On the other hand, a normal for loop is used to iterate over a sequence (like a list, tuple, string, etc.) or other iterable objects.

Here are some differences:

1. Syntax: List comprehension uses the `for` keyword followed by an expression followed by a `in` keyword, while a normal for loop uses the `for` keyword followed by a variable name in the loop statement.

2. Performance: List comprehension is generally faster than a normal for loop because they are optimized for performance.

3. Readability: List comprehension is more readable and easier to understand than a normal for loop.

4. Use Cases: List comprehension is generally used when you want to create a new list based on an existing list. For example, you can use list comprehension to create a new list that contains the squares of all 

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🛠️ Generated Python Code:

[INST] Write Python code to Explain what this function does: def bitwise_and(x, y): return x

🧑‍💻 Ask your Python question (or type 'exit' to quit):
> Write Python code to Explain what this function does: def triple(x): return x * 3


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🛠️ Generated Python Code:

[INST] Write Python code to Explain what this function does: def bitwise_and(x, y): return x

🧑‍💻 Ask your Python question (or type 'exit' to quit):
> exit

👋 Goodbye! Have a great day.
